In [ ]:
import cv2
import sklearn
import numpy as np
from pathlib import Path
import collections
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Specify where to find our data
DATA_DIR = Path("./trafficsign_data/train")

if not DATA_DIR.exists():
    print("Data path incorrect!")

In [ ]:
Imageset = collections.namedtuple("Imageset", ["paths", "labels1", "labels2"])

# Creates a tuple containing the image paths with corresponding labels
def find_images():
    paths = []
    labels1 = []
    labels2 = []

    for directory in DATA_DIR.glob("./*/*/"): # List directories 2 levels down
        for file in directory.glob("*.png"):
            paths.append(str(file))
            labels1.append(directory.parent.stem)
            labels2.append(directory.stem)

    return Imageset(np.array(paths), np.array(labels1), np.array(labels2))

In [ ]:
imgs = find_images()

In [ ]:
# Example of content of paths, labels1, labels2
print(np.vstack([imgs.paths, imgs.labels1, imgs.labels2]).T[0:1000:100])

In [ ]:
# Lets visualise a few examples

# For this tutorial, lets work with a subset
labels2_in_scope = ["F19", "F49", "F50", "F87"]

for label2 in labels2_in_scope:
    image_paths = imgs.paths[imgs.labels2 == label2]
    
    print('Examples for ' + label2)
    plt.figure()
    for i in range(4):
        plt.subplot(141 + i) # 1 row, 4 columns, <nr of image that will be provided next>
        image = cv2.imread(image_paths[i])
        plt.imshow(image[:,:,::-1]) # Convert OpenCV BGR channels to RGB
    plt.show()

In [ ]:
def print_imgs(images, channels='BGR'):
    num_images = len(images)
    plt.figure()
    for i in range(num_images):
        plt.subplot(101 + 10 * num_images + i)
        image = images[i]
        if channels == 'BGR':
            plt.imshow(image[:,:,::-1]) # Convert OpenCV BGR channels to RGB
        elif channels == 'gray':
            plt.imshow(image, cmap='gray')
        else:
            raise ValueError("Invalid channels value: " + channels)

easy_examples = [
    "./trafficsign_data/train/squares/F19/02202_12813.png",
    "./trafficsign_data/train/squares/F49/00187_05053.png",
    "./trafficsign_data/train/squares/F50/02206_13023.png",
    "./trafficsign_data/train/squares/F87/02441_13274.png"
]
hard_examples = [
    "./trafficsign_data/train/squares/F19/01713_06560.png",
    "./trafficsign_data/train/squares/F49/02102_09354.png",
    "./trafficsign_data/train/squares/F50/00900_12409.png",
    "./trafficsign_data/train/squares/F87/02060_03989.png"
]

print_imgs([cv2.imread(img) for img in easy_examples])
print_imgs([cv2.imread(img) for img in hard_examples])

In [ ]:
# Resizing
print_imgs([cv2.resize(cv2.imread(p), (36, 36)) for p in easy_examples])

In [ ]:
# Cropping
def transform(path):
    img = cv2.imread(path)
    (h, w, _) = img.shape
    transformed_img = img[0:int(h*3/4), :]
    return cv2.resize(transformed_img, (36, 36))

print_imgs([transform(p) for p in easy_examples])

In [ ]:
# Occlusion
def transform(path):
    img = cv2.imread(path)
    (h, w, _) = img.shape
    img[:, 0:int(w/4), :] = 0
    return cv2.resize(img, (36, 36))

print_imgs([transform(p) for p in easy_examples])

In [ ]:
# Mirroring
def transform(path):
    img = cv2.imread(path)
    transformed_img = cv2.flip(img, 1) # 1 = Horizontal flip
    return cv2.resize(transformed_img, (36, 36))

print_imgs([transform(p) for p in easy_examples])

In [ ]:
# Rotation
def transform(path, angle):
    img = cv2.imread(path)
    (h, w, _) = img.shape
    matrix = cv2.getRotationMatrix2D((h/2, w/2), angle, 1)
    transformed_img = cv2.warpAffine(img, matrix, (w, h))
    return cv2.resize(transformed_img, (36, 36))

print_imgs([transform(p, 10) for p in easy_examples])

In [ ]:
# Affine transform
def transform(path, angle):
    img = cv2.imread(path)
    (h, w, _) = img.shape
    matrix = cv2.getAffineTransform(
        np.float32([[0, 0], [0, w], [h, w]]),
        np.float32([[0, 0], [0, w], [h, w*2/3]])
    )
    transformed_img = cv2.warpAffine(img, matrix, (w, h))
    return cv2.resize(transformed_img, (36, 36))

print_imgs([transform(p, 5) for p in easy_examples])

In [ ]:
# HSV colorspace
def transform(path):
    img = cv2.imread(path)
    transformed_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    value = np.int16(transformed_img[:,:,2])
    value += 50
    transformed_img[:,:,2] = np.uint8(np.clip(value, 0, 255))
    transformed_img = cv2.cvtColor(transformed_img, cv2.COLOR_HSV2BGR)
    return cv2.resize(transformed_img, (36, 36))

print_imgs([transform(p) for p in easy_examples])

In [ ]:
# Histogram equalization
def transform(path):
    img = cv2.imread(path)
    transformed_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return cv2.resize(transformed_img, (36, 36))

print_imgs([transform(p) for p in easy_examples], 'gray')

def transform(path):
    img = cv2.imread(path)
    transformed_img = cv2.equalizeHist(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY))
    return cv2.resize(transformed_img, (36, 36))

print_imgs([transform(p) for p in easy_examples], 'gray')

In [ ]:
# Blurring
def transform(path):
    img = cv2.imread(path)
    transformed_img = cv2.blur(img,(4, 4))
    return cv2.resize(transformed_img, (36, 36))

print_imgs([transform(p) for p in easy_examples])

# Order of operations matters!
def transform(path):
    img = cv2.imread(path)
    return cv2.blur(cv2.resize(img, (36, 36)),(4, 4))

print_imgs([transform(p) for p in easy_examples])

In [ ]:
# Erosion
print("Erosion")
def transform(path):
    img = cv2.imread(path)
    kernel = np.ones((5,5),np.uint8)
    transformed_img = cv2.erode(img,kernel,iterations = 1)
    return cv2.resize(transformed_img, (36, 36))

print_imgs([transform(p) for p in easy_examples])

print("Dilation")
def transform(path):
    img = cv2.imread(path)
    kernel = np.ones((5,5),np.uint8)
    transformed_img = cv2.dilate(img,kernel,iterations = 1)
    return cv2.resize(transformed_img, (36, 36))

print_imgs([transform(p) for p in easy_examples])

print("Erosion + dilation")
def transform(path):
    img = cv2.imread(path)
    kernel = np.ones((5,5),np.uint8)
    transformed_img = cv2.dilate(cv2.erode(img,kernel,iterations = 1),kernel,iterations = 1)
    return cv2.resize(transformed_img, (36, 36))

print_imgs([transform(p) for p in easy_examples])

print('Original')
print_imgs([cv2.resize(cv2.imread(p), (36, 36))  for p in easy_examples])